# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import qgrid
import yaml

from pathlib import Path

from cleanup import DupInterface, grid_from_yaml
from cleanup import SizeSorter
from cleanup import stat_df
from cleanup.df import utils

# Load

In [ ]:
yaml_path = Path(r'jsl\jsl.yaml')
yaml_path = Path(r'C:\Users\lanca_000\Documents\Software\Python\Picture Cleanup\jsl\PhotoOrg-JDL.yaml')

df = utils.load_yaml(yaml_path)

# Sort

In [ ]:
ss = SizeSorter.from_yaml(yaml_path, df)
ss.flat_process()
'Bad' if (ss.mask_u == ss.mask_d).any() else 'Good'

In [ ]:
with Path(yaml_path).open('r') as file:
    cfg = yaml.load(file, Loader=yaml.SafeLoader)

bool_cols = ss.df.select_dtypes(bool).columns.to_list()
cols = cfg['default_columns'] + bool_cols

g = grid_from_yaml(yaml_path, ss.df[cols + bool_cols])
dups = ss.df.duplicated(['st_size', 'shortname', 'suffix'], keep=False)
g.df = ss.df[dups]
display(g)

In [ ]:
g.df = ss.df[dups].sort_values(['st_size', 'pathdate'], ascending=True)
temp_df = g.get_changed_df()
idx = temp_df[temp_df['mask_u']].index
g.change_selection(idx)

# QGrid

In [ ]:
di = DupInterface.from_yaml(yaml_path)
di._df = df
di.render()
display(di.widget)